In [1]:
import os

In [3]:
%pwd  ##current working directory

'd:\\Project\\redwine_end-to-end\\research'

In [4]:
os.chdir('../')  ## going ine directory back

In [5]:
%pwd

'd:\\Project\\redwine_end-to-end'

In [ ]:
from dataclasses import dataclass ##entity
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [7]:
from src.constants import *
from src.utils.common import read_yaml_file,create_directories



In [ ]:
class ConfigurationManager:  ## constant
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        self.schema = read_yaml_file(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [9]:
import os
import zipfile
import urllib.request as request
from src.logging import logger

In [ ]:
class DataIngestion:   ##compomnents
    def __init__(self, config: DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} has beeen created \n{headers}")
        else:
            logger.info("file alread exist")
            
    def get_unzip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_file:
            zip_file.extractall(unzip_path)

In [13]:
try:                                                        ##pipeline
    # Step 1: Initialize ConfigurationManager
    confi = ConfigurationManager()
    
    # Step 2: Get data ingestion config
    data_ingestion_config = confi.get_data_ingestion_config()
    
    # Step 3: Initialize DataIngestion with the config
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Step 4: Download the file
    data_ingestion.download_file()
    
    # Step 5: Extract the zip file
    data_ingestion.get_unzip_file()  # ✅ Correct method name

except Exception as ex:
    raise ex  # ✅ Correct exception handling


[2025-02-19 10:47:58,404]: INFO: common:  yaml file config\config.yaml has loaded succesfully
[2025-02-19 10:47:58,407]: INFO: common:  yaml file params.yaml has loaded succesfully
[2025-02-19 10:47:58,408]: INFO: common:  yaml file schema.yaml has loaded succesfully
[2025-02-19 10:47:58,412]: INFO: common:  path has been created in artifacts
[2025-02-19 10:47:58,416]: INFO: common:  path has been created in artifacts/data_ingestion
[2025-02-19 10:48:00,115]: INFO: 3524693216:  artifacts/data_ingestion/data.zip has beeen created 
Connection: close
Content-Length: 25115
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8bdd43e99a0542a72428bf6002b29f7e97c618e9585f42d63c7c1a31e6c3f7ae"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: C036:5C61F:2C7595:3D2BBE:67B56987
Accept-Ranges: bytes
Date: We